Rigid transformations, coordinates and reference frames
=======================================================

.. warning:: This module is in development and has not been released.

The [geometry](api/kineticstoolkit.geometry.rst) module simplifies the calculation of linear algebric operations on
series of points, vectors and transformation matrices. Contrarily to most other modules, this module does not work
with TimeSeries directly, but instead with the TimeSeries data. The convention for dimension order is strict but
well defined.

Dimension convention
--------------------

In this module, every point, vector or matrix is considered as a **series**, even constants which are series of 1. The dimensions of these series are:

- Dimension 0 = time (length of m). Constants are expressed as series with m = 1 and are broadcasted to match other series with m > 1.

- Dimension 1 (optional) = axis or row (length of 4).

- Dimension 2 (optional) = point/vector index or column (length of n).

To understand better how to express series of scalars, vectors, points, sets and matrices, please refer to the following pictures:

### Series of scalars ###

![Series of m scalars](_static/geometry/series_of_m_scalars.png){ width=400px }

### Series of vectors and points ###

![Series of m vectors](_static/geometry/series_of_m_vectors.png){ width=400px } ![Series of m points](_static/geometry/series_of_m_points.png){ width=400px }

Note the difference between vectors and points: vectors (e.g., forces, velocities) have zeros as their fourth coordinate while points (e.g., marker coordinates) have ones. This difference has an impact on algeabric calculation:

- A rigid transformation of a vector gives a new rotated vector expressed as a new orientation and amplitude.
- A rigid transformation on a point yields a translated point expressed as its new coordinates in the new reference frame.

### Series of sets of vectors or points ###

![Series of m sets of n vectors](_static/geometry/series_of_m_sets_of_n_vectors.png){ width=400px } ![Series of m sets of n points](_static/geometry/series_of_m_sets_of_n_points.png){ width=400px }

### Series of transformation matrices ###

![Series of m transformation matrices](_static/geometry/series_of_m_transformation_matrices.png){ width=500px }

### Working with constants ###

As a reminder, in the geometry module, everything is considered as a series. Therefore, always ensure that the first dimension of any array is reserved to time. For example, the vector (x = 1, y = 2, z = 3) must be expressed as `np.array([[1.0, 2.0, 3.0, 0.0]])` (note the double brackets). A common error would be to express it as `np.array([1.0, 2.0, 3.0, 0.0])` (single brackets), which would mean a series of 4 floats instead of one constant vector.

In [1]:
import kineticstoolkit.lab as ktk
import numpy as np

Matrix multiplication
-------------------------

To facilitate the multiplication of series, kineticstoolkit.geometry provides a `matmul` function that matches both matrices' time dimensions before applying the numpy's @ or * operator accordingly on each time iteration. Therefore, when dealing with series of floats, vectors, points or matrices, it is advisable to use kineticstoolkit.geometry's `matmul` instead of the @ operator, which will garranty to follow Kinetics Toolkit's array convention. For example:

**Matrix multiplication between a matrix and a series of points**

In [16]:
# A single rigid transformation matrix
T = np.array([[[1, 0, 0, 0],
               [0, 1, 0, 0],
               [0, 0, 1, 0],
               [0, 0, 0, 1]]])

# times a series of 3 points
points = np.array([[0, 1, 0, 1],
                   [1, 1, 0, 1],
                   [2, 1, 0, 1]])

ktk.geometry.matmul(T, points)  # gives a series of 3 points

array([[0., 1., 0., 1.],
       [1., 1., 0., 1.],
       [2., 1., 0., 1.]])

**Multiplication between a series of floats and a series of vectors**

In [17]:
# A series of 3 floats
floats = np.array([0., 0.5, 1.])

# times a series of 3 vectors
vectors = np.array([[0, 1, 0, 0],
                    [1, 1, 0, 0],
                    [2, 1, 0, 0]])

ktk.geometry.matmul(floats, vectors)  # gives a series of 3 vectors

array([[0. , 0. , 0. , 0. ],
       [0.5, 0.5, 0. , 0. ],
       [2. , 1. , 0. , 0. ]])

**Dot product between a series of points and a single point**

In [18]:
# A series of 3 points
points = np.array([[0, 1, 0, 1],
                   [1, 1, 0, 1],
                   [2, 1, 0, 1]])

# times a single point
point = np.array([[2, 3, 4, 1]])

ktk.geometry.matmul(points, point)  # gives a series of 3 scalars

array([4., 6., 8.])

Creating series of rotation matrices
------------------------------------
The function `kineticstoolkit.geometry.create_rotation_matrices` allows creating series of Nx4x4 matrices around a given axis. For example, this command creates a rotation matrix of 90 degrees around the x axis. Note that the angle argument is a list or an array, to comply with the `kineticstoolkit.geometry`'s convention above.

In [5]:
ktk.geometry.create_rotation_matrices('x', [np.pi/2])

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.]]])

As another example, this will create a series of 100 rotation matrices around the z axis, from 0 to
360 degrees:

In [6]:
ktk.geometry.create_rotation_matrices('z', np.linspace(0, 2 * np.pi, 100))

array([[[ 1.        , -0.        ,  0.        ,  0.        ],
        [ 0.        ,  1.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  1.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],

       [[ 0.99798668, -0.06342392,  0.        ,  0.        ],
        [ 0.06342392,  0.99798668,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  1.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],

       [[ 0.99195481, -0.12659245,  0.        ,  0.        ],
        [ 0.12659245,  0.99195481,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  1.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],

       ...,

       [[ 0.99195481,  0.12659245,  0.        ,  0.        ],
        [-0.12659245,  0.99195481,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  1.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.

Creating series of reference frames
-----------------------
Let's say we have the position of three markers, and we want to create a reference frame based on these markers. This is the aim of the `create_reference_frames` function.

If the markers are at these positions (0, 0, 0), (1, 0, 0) and (0, 1, 0):

In [7]:
global_markers = np.array(
    [[[0., 1., 0.],
      [0., 0., 1.],
      [0., 0., 0.],
      [1., 1., 1.]]])

Now we can create a reference frame based on these three markers. Please consult the [API reference](api/kineticstoolkit.geometry.rst) for the different marker configurations available for the creation of reference frames.

In [8]:
T = ktk.geometry.create_reference_frames(global_markers)

T

array([[[-0.70710678,  0.70710678,  0.        ,  0.33333333],
        [-0.70710678, -0.70710678,  0.        ,  0.33333333],
        [ 0.        ,  0.        ,  1.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]])

Calculating local coordinates
---------------------
We now have a reference frame defined from three markers. If we are interested to know the local position of these markers in the new-created frame, we can use the function `get_local_coordinates`.

In [9]:
local_markers = ktk.geometry.get_local_coordinates(global_markers, T)

local_markers

array([[[ 0.47140452, -0.23570226, -0.23570226],
        [ 0.        ,  0.70710678, -0.70710678],
        [ 0.        ,  0.        ,  0.        ],
        [ 1.        ,  1.        ,  1.        ]]])

Calculating global coordinates
----------------------
In the case we have the markers' local coordinates and we would like to express these markers in the global reference frame, we would use the mirror function `get_global_coordinates`.

In [10]:
ktk.geometry.get_global_coordinates(local_markers, T)

array([[[0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 0.],
        [1., 1., 1.]]])

in which case the results is effectively the global points as we defined them at the beginning.

For more information on geometry, please check the [API Reference for the geometry module](api/kineticstoolkit.geometry.rst).